# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [3]:
import math

import numpy as np
import pandas as pd

Загрузим данные и проведем предобработку данных как на семинаре.

In [4]:
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder

In [5]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

 61% 5.00M/8.20M [00:00<00:00, 49.4MB/s]
100% 8.20M/8.20M [00:00<00:00, 68.5MB/s]
Archive:  articles-sharing-reading-from-cit-deskdrop.zip
  inflating: articles/shared_articles.csv  
  inflating: articles/users_interactions.csv  


In [6]:
articles_df = pd.read_csv("articles/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [7]:
interactions_df = pd.read_csv("articles/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [8]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [9]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
    "VIEW": 1.0,
    "LIKE": 2.0,
    "BOOKMARK": 2.5,
    "FOLLOW": 3.0,
    "COMMENT CREATED": 4.0,
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(
    lambda x: event_type_strength[x]
)

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [10]:
users_interactions_count_df = (
    interactions_df.groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId")
    .size()
)
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[
    users_interactions_count_df >= 5
].reset_index()[["personId"]]
print("# users with at least 5 interactions:", len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [11]:
interactions_from_selected_users_df = interactions_df.loc[
    np.in1d(interactions_df.personId, users_with_enough_interactions_df)
]

In [12]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглаживаем полученный результат, взяв от него логарифм.

In [13]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = (
    interactions_from_selected_users_df.groupby(["personId", "contentId"])
    .eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index()
    .set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = interactions_from_selected_users_df.groupby(
    ["personId", "contentId"]
)["timestamp"].last()

interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Почистим данные от лишних статей

In [14]:
commonContentIds = set(interactions_full_df['contentId']) & set(articles_df['contentId'])

In [15]:
interactions_full_df.drop(interactions_full_df[~interactions_full_df['contentId'].isin(commonContentIds)].index, inplace=True)

In [16]:
articles_df.drop(articles_df[~articles_df['contentId'].isin(commonContentIds)].index, inplace=True)

Разобьём выборку на обучение и контроль по времени.

In [17]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp < split_ts
].copy()
interactions_test_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp >= split_ts
].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29302
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Уберем пользователей которые есть в тесте, но нет в трейне, так как для них иначе будет холодный старт

In [18]:
interactions_test_df.drop(interactions_test_df[~interactions_test_df['personId'].isin(interactions_train_df['personId'])].index, inplace=True)

Сделаем маппинг id'шников (необходимо для предикта в LightFM)

In [19]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

interactions_train_df['personId_enc'] = labelencoder.fit_transform(interactions_train_df['personId'])
interactions_test_df['personId_enc'] = labelencoder.transform(interactions_test_df['personId'])

articles_df['contentId_enc'] = labelencoder.fit_transform(articles_df['contentId'])
interactions_train_df['contentId_enc'] = labelencoder.transform(interactions_train_df['contentId'])
interactions_test_df['contentId_enc'] = labelencoder.transform(interactions_test_df['contentId'])

Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [20]:
interactions = (
    interactions_train_df.groupby("personId_enc")["contentId_enc"]
    .agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId_enc": "true_train"})
    .set_index("personId_enc")
)

interactions["true_test"] = interactions_test_df.groupby("personId_enc")["contentId_enc"].agg(
    lambda x: list(x)
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    ""
    for x in range(
        len(interactions.loc[pd.isnull(interactions.true_test), "true_test"])
    )
]

interactions.head(1)

,true_train,true_test
personId_enc,,
0,"[696, 1214]","[980, 1561, 2615, 2864]"


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [21]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 KB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp39-cp39-linux_x86_64.whl size=875594 sha256=9d100628d9d508bd160972bce630706d592b7c5d4e5e6cd122ea0e6153f5c248
  Stored in directory: /root/.cache/pip/wheels/d8/65/93/6ac8180274dc2e8f86ff326be62da1dfa55dc158fd45faba7d
Successfully built lightfm


In [22]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1 (1.5 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [23]:
from scipy.sparse import csr_matrix

def create_sparse_matrix(interactions_df, n_users, n_articles):    
    sparse_matrix = pd.DataFrame(0, index=n_users, columns=n_articles)
    
    
    user_ids = interactions_df['personId_enc'].values
    content_ids = interactions_df['contentId_enc'].values
    eventStrengths = interactions_df['eventStrength'].values
    
    
    for i in range(len(interactions_df)):
        sparse_matrix.loc[user_ids[i], content_ids[i]] = eventStrengths[i] if eventStrengths[i] is not np.nan else 0
    
    return csr_matrix(sparse_matrix.values)

In [24]:
n_users = np.unique(interactions_train_df['personId_enc'])
n_articles = np.unique(articles_df['contentId_enc'])

In [25]:
data_train = create_sparse_matrix(interactions_train_df, n_users, n_articles)
data_test = create_sparse_matrix(interactions_test_df, n_users, n_articles)

In [26]:
data_train

<1112x2976 sparse matrix of type '<class 'numpy.float64'>'
	with 29302 stored elements in Compressed Sparse Row format>

In [27]:
data_test

<1112x2976 sparse matrix of type '<class 'numpy.float64'>'
	with 9418 stored elements in Compressed Sparse Row format>

## Задание 2 (0.5 балла)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [47]:
model = LightFM(loss='warp', random_state=42)
model.fit(data_train, epochs=20);

In [333]:
precision_at_k(model, data_test, data_train, k=10).mean()

0.0074423486

Низкое качество

## Задание 3 (2 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [28]:
# Склеим текст и заголовок
feat = articles_df[['title', 'text']]
feat.index = articles_df['contentId_enc']
feat = feat.reindex(n_articles).fillna('')
feat = feat.apply(lambda x: x['title'] + ' ' + x['text'], axis=1)
feat


contentId_enc
0       Google unleashes DeepMind on energy-hungry dat...
1       Indústria 4.0: desafios e oportunidades *Igor ...
2       12 JavaScript Hacks In this post I will share ...
3       Australian Bitcoin Entrepreneur Launches Robo-...
4       React Native v0.32.0-rc.0 released v0.32.0-rc....
                              ...                        
2971    Baidu abre laboratório de realidade aumentada ...
2972    Better Exposed Filters  The Better Exposed Fil...
2973    Elasticsearch: CSV exporter for Kibana Discove...
2974    Quer reclamar? Desenvolvedores vencem hackatho...
2975    Este é o melhor jeito de entender como as mulh...
Length: 2976, dtype: object

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

In [335]:
# учимся только на статьях из трейна
vectorizer.fit(feat)

feat_tranform = vectorizer.transform(feat)

In [336]:
model.fit(data_train, item_features=feat_tranform, epochs=20);

In [337]:
precision_at_k(model, data_test, data_train, k=10, item_features=feat_tranform).mean()

0.0076519917

Качество улучшилось

## Задание 4 (1.5 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [29]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=c47e7b6e7d8b450efa9fec3fe9ea3af06717feab0b1ec927d67f56be8f67a1fc
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [30]:
import regex
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Посмотрим какие есть языки

In [31]:
np.unique(articles_df['lang'], return_counts=True)

(array(['en', 'es', 'ja', 'la', 'pt'], dtype=object),
 array([2148,    2,    2,    2,  822]))

In [32]:
stopwords.fileids()

['arabic',
 'azerbaijani',
 'basque',
 'bengali',
 'catalan',
 'chinese',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hebrew',
 'hinglish',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

nltk предлагает возможность взять готовый набор стоп-слов для английского, португальского и испанского. Латынь и японский за неимением готового набора стоп-слов и малым количеством наблюдений остаются без стоп слов и очищаются только от пунктуации.

In [33]:
m = MorphAnalyzer()

def words_only(text):
    try:
        return regex.findall(r'[\p{L}]+', text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

def remove_stopwords(lemmas, stopwords):
    return [w for w in lemmas if not w in stopwords]

def clean_text(text, stopwords):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas, stopwords))

In [34]:
en_stopwords = stopwords.words('english')
pt_stopwords = stopwords.words('portuguese')  
es_stopwords = stopwords.words('spanish')

In [35]:
process_feat = {}

for key in feat.keys():
    info = articles_df[articles_df['contentId_enc'] == key].values

    if (len(info) == 0):
      process_feat[key] = ''
      continue

    lang = info[0][-1]

    if lang == 'en':
      process_feat[key] = clean_text(feat[key], en_stopwords)
    elif lang == 'pt':
      process_feat[key] = clean_text(feat[key], pt_stopwords)
    elif lang == 'es':
      process_feat[key] = clean_text(feat[key], es_stopwords)
    else:
      process_feat[key] = clean_text(feat[key], [])


process_feat = pd.Series(process_feat)
process_feat.index.name = 'contentId'

In [36]:
process_feat

contentId
0       google unleashes deepmind on energy hungry dat...
1       indústria desafios e oportunidades igor schiew...
2       javascript hacks in this post i will share ext...
3       australian bitcoin entrepreneur launches robo ...
4       react native v rc released v rc on github npm ...
                              ...                        
2971    baidu abre laboratório de realidade aumentada ...
2972    better exposed filters the better exposed filt...
2973    elasticsearch csv exporter for kibana discover...
2974    quer reclamar desenvolvedores vencem hackathon...
2975    este é o melhor jeito de entender como as mulh...
Length: 2976, dtype: object

In [39]:
vectorizer.fit(process_feat)

process_feat_tranform = vectorizer.transform(process_feat)

In [48]:
model.fit(data_train, item_features=process_feat_tranform, epochs=20);

In [396]:
precision_at_k(model, data_test, data_train, k=10, item_features=process_feat_tranform).mean()

0.0077568134

Улучшилось ли качество предсказания?

Качество улучшилось

## Задание 5 (1.5 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [113]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [114]:
from bayes_opt import BayesianOptimization

parameters = {
    'components_num': (20, 80),
    'epoch_num': (5, 50),
    'learning_rate': (0.01, 0.1)
}


def optimisation_func(components_num, epoch_num, learning_rate):
    model = LightFM(learning_rate=learning_rate, loss='warp',
                    no_components=int(components_num))
    model.fit(data_train, item_features=process_feat_tranform,
              epochs=int(epoch_num))

    precision = precision_at_k(model, data_test, data_train,
                               k=10, item_features=process_feat_tranform).mean()
    
    return precision
  
optimizer = BayesianOptimization(
  f = optimisation_func,
  pbounds = parameters,
  verbose = 5,
  random_state = 42, 
)

optimizer.maximize(
  init_points = 4,
  n_iter = 3, 
)

|   iter    |  target   | compon... | epoch_num | learni... |
-------------------------------------------------------------
| 1         | 0.005346  | 42.47     | 47.78     | 0.07588   |
| 2         | 0.007652  | 55.92     | 12.02     | 0.02404   |
| 3         | 0.007862  | 23.49     | 43.98     | 0.0641    |
| 4         | 0.00587   | 62.48     | 5.926     | 0.09729   |
| 5         | 0.008281  | 56.15     | 12.34     | 0.03104   |
| 6         | 0.007233  | 57.07     | 12.82     | 0.01674   |
| 7         | 0.007233  | 55.92     | 13.07     | 0.05259   |


Попробуем обучить на лучшем результате

In [125]:
best_model = LightFM(learning_rate=0.03, loss='warp', no_components=56)
best_model.fit(data_train, item_features=process_feat_tranform, epochs=12);

In [126]:
precision_at_k(best_model, data_test, data_train, k=10, item_features=process_feat_tranform).mean()

0.008385744

Подбор параметров повысил качество

Сохраним модель

In [133]:
import pickle

with open('lightFm.pkl', 'wb') as f:
    pickle.dump(best_model, f)

## Задание 6 (1 балл)

Реализуйте функции для вычисления следующих метрик:
* precision@k
* recall@k
* NDCG@k



In [136]:
def calc_precision(column):
    return (
        interactions
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()

In [128]:
def calc_recall(column):
    return (
        interactions
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            len(row[column]),
            axis=1)).mean()

In [129]:
def dcg_score(y_true, y_score, k=10, gains="exponential"):
    """Discounted cumulative gain (DCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    gains : str
        Whether gains should be "exponential" (default) or "linear".
    Returns
    -------
    DCG @k : float
    """
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")

    # highest rank is 1 so +2 instead of +1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10, gains="exponential"):
    """Normalized discounted cumulative gain (NDCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    gains : str
        Whether gains should be "exponential" (default) or "linear".
    Returns
    -------
    NDCG @k : float
    """
    best = dcg_score(y_true, y_true, k, gains)
    actual = dcg_score(y_true, y_score, k, gains)
    return actual / best

## Задание 7 (1 балл)

Вычислите значения реализованных метрик для $k=10$ для лучшей полученной модели в предыдущих шагах.

Найдите уже реализованные варианты этих метрик в библиотеках lightfm и sklearn. Сравните полученные у вас значения метрик с результатами встроенных в библиотеки метрик.

In [134]:
with open('lightFm.pkl', 'rb') as f:
    best_model = pickle.load(f)


In [137]:
def pairs(n_users, n_articles):
  users_ids = []
  articles_ids = []

  for i in n_users:
    for j in n_articles:
      users_ids.append(i)
      articles_ids.append(j)

  return users_ids, articles_ids

In [138]:
users_ids, articles_ids = pairs(n_users, n_articles)

In [151]:
predictions = best_model.predict(users_ids, articles_ids)

In [140]:
test = pd.DataFrame(data={'personId_enc': users_ids,
                          'contentId_enc': articles_ids,
                          'predictions': predictions})

In [152]:
top_k = 10
test = test.sort_values('predictions', ascending=False)
predictions_ = test.groupby('personId_enc')['contentId_enc'].aggregate(list)
tmp_predictions = []

for personId in interactions.index:
    prediction = np.array(predictions_.loc[personId])
    
    tmp_predictions.append(
        list(prediction[~np.in1d(
            prediction,
            interactions.loc[personId, 'true_train'])])[:top_k])
    
interactions['prediction_content'] = tmp_predictions

In [145]:
calc_precision('prediction_content')

0.00522802281375267

In [146]:
calc_recall('prediction_content')

0.0034172661870503595

In [147]:
precision_at_k(best_model, data_test, data_train, k=10, item_features=process_feat_tranform).mean()

0.008385744

In [148]:
from lightfm.evaluation import recall_at_k

recall_at_k(best_model, data_test, data_train, k=10, item_features=process_feat_tranform).mean()

0.009277481217449037

In [156]:
from sklearn.metrics import ndcg_score

ndcg_score([data_test.toarray().reshape(-1)], [predictions])

0.505538644097635

## Задание 8 (1 балл)

Реализуйте алгоритм ALS и примените его для решения задачи ноутбука.

**ALS**

Итак, поставлена задача построения модели со скрытыми переменными (latent factor model) для коллаборативной фильтрации:

$$ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \to \min_{P,Q}$$

Суммирование ведется по всем парам $(u, i),$ для которых известен рейтинг $r_{ui}$ (и только по ним), а $p_u, q_i$ – латентные представления пользователя~$u$ и товара $i$, соответственно, матрицы $P, Q$ получаются путем записывания по столбцам векторов $p_u, q_i$ соответственно.

Подход ALS (Alternating Least Squares) решает задачу, попеременно фиксируя матрицы $P$ и $Q$, — оказывается, что, зафиксировав одну из матриц, можно выписать аналитическое решение задачи для другой.

$$\nabla_{p_u} \bigg[ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \bigg] = \sum_{i} 2(r_{ui} - \langle p_u, q_i \rangle)q_i = 0$$

Воспользовавшись тем, что $a^Tbc = cb^Ta$, получим
$$\sum_{i} r_{ui}q_i - \sum_i q_i q_i^T p_u = 0.$$

Тогда окончательно каждый столбец матрицы $P$ можно найти по формуле
$$p_u = \bigg( \sum_i q_i q_i^T\bigg)^{-1}\sum_ir_{ui}q_i \;\; \forall u,$$

аналогично для столбцов матрицы $Q$
$$q_i = \bigg( \sum_u p_u p_u^T\bigg)^{-1}\sum_ur_{ui}p_u \;\; \forall i.$$

Таким образом мы можем решать оптимизационную задачу, поочередно фиксируя одну из матриц $P$ или $Q$ и проводя оптимизацию по второй.

**Оригинальная статья c постановкой задачи для ALS на explicit feedback:**

* Bell, R.M. and Koren, Y., 2007, October. Scalable collaborative filtering with jointly derived neighborhood interpolation weights. In Seventh IEEE international conference on data mining (ICDM 2007) (pp. 43-52). IEEE.

**Оригинальная статья с ALS для implicit данных, которая стала более известной:**

* Hu, Y., Koren, Y. and Volinsky, C., 2008, December. Collaborative filtering for implicit feedback datasets. In 2008 Eighth IEEE international conference on data mining (pp. 263-272). Ieee.


Посмотрел последнюю статью + реализация к ней кода https://www.kaggle.com/code/shivendra91/recommendation-als

In [357]:
import scipy.sparse as sp
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse.linalg import spsolve

In [358]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 42):   
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008.
    
    Параметры:
    
    training_set - Наша матрица оценок имеет форму m x n, где m — количество пользователей, а n — количество товаров.
    Должна быть разреженная матрица csr.

    lambda_val - Используется для регуляризации при чередовании метода наименьших квадратов. Увеличение этого значения может увеличить смещение
    но уменьшить дисперсию. По умолчанию 0,1.
    
    alpha - Параметр, связанный с матрицей достоверности, обсуждаемой в статье, где Cui = 1 + alpha*Rui.
    В статье установлено, что значение по умолчанию, равное 40, является наиболее эффективным. Уменьшение этого значения уменьшит изменчивость достоверности между
    различные рейтинги.

    iterations - количество итераций, которое нужно чередовать между вектором характеристик пользователя и вектором характеристик товаров в
    чередование наименьших квадратов. Больше итераций позволит улучшить сходимость за счет увеличения объема вычислений.
    Авторы обнаружили, что 10 итераций было достаточно, но для сходимости может потребоваться больше.
    
    rank_size - количество латентных представлений в векторах функций пользователя/элемента. В статье рекомендуется варьировать это
    между 20-200. Увеличение количества может привести к переобучению, но может уменьшить систематическую ошибку.
    
    seed - для воспроизводимости результатов


    Результат:

    Скалярное произведение векторов X, Y дает ожидаемый «рейтинг» в каждой паре исходной матрицы.
    '''
    conf = alpha * training_set
    num_user = conf.shape[0]
    num_item = conf.shape[1] 
    
    # случайным образом инициализируем векторы признаков X и Y
    rstate = np.random.RandomState(seed)
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size)))
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size)))

    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)

    lambda_eye = lambda_val * sparse.eye(rank_size) # регуляризации lambda*I
    
    for iter_step in range(iterations):
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        # Итерация решения для X на основе фиксированного Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() 
            pref = conf_samp.copy() 
            pref[pref != 0] = 1
            CuI = sparse.diags(conf_samp, [0]) 
            yTCuIY = Y.T.dot(CuI).dot(Y) 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) 
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
    
        # Итерация решения для Y на основе фиксированного X
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray()
            pref = conf_samp.copy()
            pref[pref != 0] = 1 
            CiI = sparse.diags(conf_samp, [0])
            xTCiIX = X.T.dot(CiI).dot(X) 
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T)
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)

    return X, Y.T

In [359]:
user_vecs, articles_vecs = implicit_weighted_ALS(data_train)

In [360]:
pd.DataFrame.sparse.from_spmatrix(user_vecs.dot(articles_vecs))

,0,1,2,3,4,5,6,7,8,9,...,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975
0,-0.033819,-0.051754,0.289484,-0.104567,0.127782,0.341565,-0.050967,0.064539,-0.183601,-0.053165,...,-0.127712,-0.063604,0.195922,0.0,-0.207078,0.0,-0.051404,-0.121037,0.141879,0.0
1,0.828852,0.017454,0.460150,0.466844,0.536716,0.534460,0.883315,-0.374976,0.580667,1.000496,...,0.954280,0.676172,0.284992,0.0,0.968526,0.0,0.787272,0.094800,-0.204800,0.0
2,1.225257,-0.133249,-0.818224,-0.070658,-0.284690,1.083036,1.103886,0.092985,0.248321,0.422853,...,0.752333,1.173604,-0.473461,0.0,1.018779,0.0,0.302377,-0.150401,-0.215762,0.0
3,0.975221,0.321642,-0.601620,0.298723,-0.146851,0.400421,-0.252896,0.112747,-0.220853,0.084852,...,0.143466,1.086192,-0.047522,0.0,0.085987,0.0,0.171423,0.160987,0.319865,0.0
4,0.380571,-0.125150,0.209017,-0.224644,0.126809,0.150761,-0.019917,0.123571,0.016332,-0.348650,...,-0.003034,-0.077572,-0.041181,0.0,0.022435,0.0,-0.193741,-0.014360,0.171383,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,0.081986,-0.163746,0.083299,-0.173212,-0.077410,0.104096,-0.015497,-0.261523,0.035880,-0.049559,...,-0.069709,0.402890,0.046293,0.0,-0.198392,0.0,-0.112769,0.336942,-0.074343,0.0
1108,0.054252,-0.158545,-0.168217,-0.012307,-0.262017,0.183427,-0.170376,-0.064096,-0.287177,-0.001490,...,0.023115,0.160785,-0.059060,0.0,0.122481,0.0,-0.102468,0.089663,0.301837,0.0
1109,0.113995,0.046530,0.441193,0.138395,0.120380,-0.037280,-0.274976,-0.483721,-0.210232,0.289794,...,0.104526,0.291366,0.583012,0.0,0.060636,0.0,0.061435,0.488738,0.438966,0.0
1110,-0.336625,0.317455,0.279168,-0.174551,0.241231,0.318668,-0.300534,-0.074034,0.139740,0.159578,...,-0.183422,0.036069,0.553433,0.0,-0.142966,0.0,-0.160011,0.678942,-0.047025,0.0
